In [1]:
!pip install -U bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.5 MB/s eta 0:00:00:00:0100:01


In [2]:
# !pip install -U numpy;
# !pip install -U transformers

In [3]:
import transformers
from transformers import (AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          AutoTokenizer,
                          TrainingArguments
                          )
import torch

In [4]:
#parameters
quant_config = BitsAndBytesConfig(use_4bit=True,
                                  bnb_4bit_compute_dtype=torch.bfloat16,
                                  bnb_4bit_quant_type='nf4')
# model_name = "NousResearch/Llama-2-7b-hf"
model_name = "haoranxu/ALMA-7B"


Unused kwargs: ['use_4bit']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


## Load pretrained

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quant_config,
                                             device_map='auto')

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

#### Training arguments

## Data


In [7]:
!git clone https://github.com/wmt-conference/wmt21-news-systems.git

Cloning into 'wmt21-news-systems'...
remote: Enumerating objects: 970, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 970 (delta 60), reused 58 (delta 36), pack-reused 885 (from 1)
Receiving objects: 100% (970/970), 108.27 MiB | 18.54 MiB/s, done.
Resolving deltas: 100% (495/495), done.
Updating files: 100% (817/817), done.


In [8]:
src_path = "/kaggle/working/wmt21-news-systems/txt/references/newstest2021.de-en.ref.A.en"
ref_path = "/kaggle/working/wmt21-news-systems/txt/sources/newstest2021.de-en.src.de"


In [9]:
source_str = open(src_path).read()
ref_str = open(ref_path).read()
source_str

"Biker reacts lightning fast and prevents worse.\nA traffic accident occurred on Friday evening on the country road between Machtolsheim and Merklingen where a bike rider was only slightly injured thanks to his quick reaction.\nA female car driver overlooks a biker\nAround 21:30 a 44 year old female car driver, coming from Merklingen with her VW, wanted to turn left into the country road 1230 into the direction of Machtolsheim.\nAt this, she overlooked the 24 year old motorbike driver with right-of way who came from the direction of Nellingen.\nWhen they collided hand and foot brake pedals break off\nThe biker still attempted to evade the car, however, brushed against the car at the rear end.\nThrush the brushing against the car hand break handle and foot break pedal were ripped off the Suzuki and the driver was no longer able to break.\nAlerted he steered his bike into the grass strip beside the road and taxied it to a standstill without losing control.\nBiker suffers leg injuries\nTh

In [10]:
PROMPT_TEMPLATE = "{source}: {source_sentence} {dest}: "
PROMPT_TEMPLATE = "[{source}]: {source_sentence} Translate to [{dest}]: "
PROMPT_TEMPLATE = "{source}: {source_sentence} \n Übersetz nach {dest}: "

In [11]:
!pip install datasets

from datasets import Dataset
def build_parallel_ds(source_str,ref_str):
  ref_list = ref_str.split("\n")
  src_list = source_str.split("\n")
  assert len(ref_list) == len(src_list)
  ds = {"src":[],"ref":[],"prompt":[]}
  for i in range(len(ref_list)):
    ds["src"].append(src_list[i])
    ds["ref"].append(ref_list[i])
    prompt = PROMPT_TEMPLATE.format(
        source="en",
        dest="de",
        source_sentence=src_list[i])
    ds["prompt"].append(prompt)

  return Dataset.from_dict(ds)

In [12]:
!pip install unbabel-comet

from comet import download_model, load_from_checkpoint

comet_path = download_model("Unbabel/wmt20-comet-da")
comet_model = load_from_checkpoint(comet_path)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.6.1
    Uninstalling torchmetrics-1.6.1:
      Successfully uninstalled torchmetrics-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


## Zero-shot : COMET evaluationn

In [13]:
import numpy as np

def eval_comet_zeroshot(parallel_ds, model, tokenizer, comet_model, prompt=PROMPT_TEMPLATE):

  scores = []
  def translate(batch):
    prompts =  [prompt.format(
        source_sentence=batch["src"])]

    input_ids = tokenizer(prompts,padding=True,padding_side="left").input_ids # Return PyTorch tensors
    ref_toks = tokenizer(batch["ref"]).input_ids

    #non -batched part :

    length = len(input_ids[0])
    max_length_ref = np.max([len(iids) for iids in ref_toks])
    input_idss = torch.tensor(input_ids, device="cuda")

    with torch.no_grad():
      output_tokens = model.generate(input_idss,
                                      max_new_tokens=int(3*max_length_ref),
                                     num_beams=2,
                                     early_stopping=True,
                                      )
    output = tokenizer.batch_decode(output_tokens[:,length:],batch_size=len(batch))
    # output = [o.split("\n")[0] for o in output] 
    return output

  batched_ds = parallel_ds.batch(1)
  i = 0
  for batch in batched_ds :
    mt = translate(batch)
    pred = [{"src" : src, "ref": ref, "mt" : mt_.replace(tokenizer.pad_token,comet_model.encoder.tokenizer.pad_token)} for src,ref,mt_ in zip(batch["src"],batch["ref"],mt)] 
    
    scores.append(comet_model.predict(pred,batch_size=1))

    print(pred)
    i+= 1
  return scores


In [14]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

torch.cuda.empty_cache()
# 

In [15]:
PROMPT_TEMPLATE_SIMPLE = "en: {source_sentence} de: "
PROMPT_TEMPLATE_SIMPLE_BRACK = "[en]: {source_sentence} [de]: "

PROMPT_TEMPLATE_EN = "English: {source_sentence} Translate to German: "
PROMPT_TEMPLATE_EN_BRACK = "[en]: {source_sentence} Translate to [de]: "
PROMPT_TEMPLATE_DE = "Englisch: {source_sentence} \n Übersetze ins Deutsche: "
PROMPT_TEMPLATE_DE_BRACK = "[en]: {source_sentence} \n Übersetze ins [de]: "

prompt_list = [
PROMPT_TEMPLATE_SIMPLE,
PROMPT_TEMPLATE_SIMPLE_BRACK,
PROMPT_TEMPLATE_EN,
PROMPT_TEMPLATE_EN_BRACK,
PROMPT_TEMPLATE_DE,
PROMPT_TEMPLATE_DE_BRACK,
]


In [ ]:
ds = build_parallel_ds(source_str, ref_str).shuffle().take(30)
scores = []
for p in prompt_list:
    score = eval_comet_zeroshot(ds, model, tokenizer, comet_model,prompt=p)
    scores.append(score)

Batching examples:   0%|          | 0/30 [00:00<?, ? examples/s]

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  6.50it/s]


[{'src': 'The fire broke out in three places in the church, which were at a considerable distance from one another.', 'ref': 'Das Feuer war an drei weit voneinander entfernten Stellen in der Kirche ausgebrochen.', 'mt': '3 Plätze im Gotteshaus brannten gleichzeitig aus, die sich von einander entfernt befanden.</s>'}]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 15.06it/s]


[{'src': 'From July 31 on the 33 super sports cars will race through the Lausitz.', 'ref': 'Ab dem 31. Juli werden die 33 Supersportwagen durch die Lausitz rasen.', 'mt': '33 Supersportwagen werden ab dem 31. Juli in der Lausitz um die Wette fahren.</s>'}]


...

[{'src': 'Beside that, Corona testing should be performed in emergency shelters.', 'ref': 'Außerdem solle in Notunterkünften auf Corona getestet werden.', 'mt': '‚Nebendem sollten Corona-Tests in Notunterkünften durchgeführt werden.‘</s>'}]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 15.20it/s]


[{'src': 'The respective person as well as all other people in Kaesong who were in contact this person had been quarantined.', 'ref': 'Die betroffene Person sowie alle Menschen in Kaesong, die mit ihr Kontakt gehabt hätten, seien unter Quarantäne gestellt worden.', 'mt': '\x93Der jeweilige Mensch sowie alle anderen Menschen in Kaesong, die mit ihm in Kontakt waren, wurden in Quarantäne genommen.\x94</s>'}]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 15.31it/s]


[{'src': '“Even if they don’t scream as loudly as big babies, of course.”', 'ref': '"Auch wenn sie natürlich nicht ganz so laut schreien wie große Babys".', 'mt': '‚‚Auch wenn sie nicht so laut schreien wie große Babys, natürlich‘‘</s>'}]


In [ ]:
print(score)

In [ ]:
comet_model.encoder.tokenizer.encode(comet_model.encoder.tokenizer.pad_token)
comet_model.encoder.tokenizer.decode([0,1,2])

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(list(map(lambda x: x[0],scores)))
#COMET 20

In [ ]:
print(list(map(lambda x: x[0],scores)))
#beam search=5

In [ ]:
print(list(map(lambda x: x[0],scores)))
#beam search=2